# Debugging and Experiments

In [1]:
from data_distillation.models.transformer.feature_extractors.triplet_cross_attention_vit import TripletCrossAttentionViT as TCAiT
from data_distillation.models.transformer.feature_extractors.tcait_extractor import TCAiTExtractor
from data_distillation.models.transformer.feature_extractors.pyramid.pyra_tcait import PyraTCAiT
from data_distillation.models.transformer.feature_extractors.pyramid.pyramid_vision_transformer import PyramidVisionTransformer as PVT

from data_distillation.losses.triplet_losses.triplet_classification_loss import TripletClassificationLoss as TCLoss
from data_distillation.losses.triplet_losses.triplet_loss import TripletLoss

from data_distillation.optimization.schedulers.warmup_cosine_scheduler import WarmupCosineScheduler

from data_distillation.testing.data.test_triplets import TestTriplets
from data_distillation.testing.data.triplets import Triplets
from data_distillation.data_distiller import DataDistiller

from torch.utils.data.dataloader import DataLoader

import torch.optim as optim
import torch.nn as nn
import torch

from torchvision.transforms import Compose, Resize, RandomResizedCrop, CenterCrop, RandomHorizontalFlip, ColorJitter, Normalize, Lambda

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

/home/hice1/cclark339/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Debugging

### `DataDistiller` Object

1. Initialization **(DONE)**
2. Initial Embedding **(DONE)**
3. Omegas Scheduling **(DONE)**
4. Triplet Mining **(DONE)**
5. Saving/Loading Embeddings **(DONE)**
6. Saving/Loading Triplets **(DONE)**
7. Main Training/Validation Loop

In [2]:
# paths
BASE_DATA_DIR = '/home/hice1/cclark339/scratch/Data/WildlifeReID-10K/'
TRAIN_META = BASE_DATA_DIR + 'train_metadata.csv'
VALID_META = BASE_DATA_DIR + 'valid_metadata.csv'

CHECKPOINTS_DIR = '/home/hice1/cclark339/scratch/Checkpoints/Experimentation/'
EMBEDDINGS_DIR = '/home/hice1/cclark339/scratch/Embeddings/Experimentation/'
TRIPLETS_DIR = '/home/hice1/cclark339/scratch/Triplets/Experimentation/'

# transform args
TRAIN_DIM = 224
VALID_DIM = 256

BRIGHTNESS = 0.4
CONTRAST = 0.4
SATURATION = 0.4
HUE = 0.1

NORM_MEANS = [0.485, 0.456, 0.406]
NORM_STDS = [0.229, 0.224, 0.225]

# setup args
BATCH_SIZE = 16
NUM_CLASSES = 7747
CHANNELS = 3
NUM_EPOCHS = 10
DEVICE = 'gpu'
GPU_ID = 0
START_EPOCH = 0
USE_DDP = False
DISABLE_PROGRESS_BAR = False

# model args
EMBED_DIMS = [64, 128, 320, 512]
HEAD_COUNTS = [1, 2, 5, 8]
MLP_RATIOS = [8, 8, 4, 4]
SR_RATIOS = [8, 4, 2, 1]
DEPTHS = [3, 4, 6, 3]
PATCH_SIZE = 4
NUM_STAGES = 4
DROPOUT = 0.1
USE_IMPROVED = True
CLASSIFICATION_INTENT = False
ADD_CLASSIFIER = True

# optimizer and scheduler args
USE_LABEL_SMOOTHING = False
LEARNING_RATE = 1e-4
BETAS = [0.9, 0.999]
WEIGHT_DECAY = 2.5e-4
PATIENCE = 10
WARMUP_EPOCHS = 3
ETA_MIN = 0.0

# triplet mining args
NUM_TRIPLETS = 20
NUM_WORKERS = 1
THETAS = [0.3, 0.6]
PRETR_MODEL = 'tf_efficientnet_b5.ns_jft_in1k'
MAX_ATTEMPTS = 100
P_MAX = 1.0
MARGIN = 1.0
REMINE_FREQ = 5

In [3]:
train_df = pd.read_csv(TRAIN_META)[['identity', 'path']].iloc[:1600]
train_df.head()

,identity,path
0,AAUZebraFish_5,images/AAUZebraFish/data/Vid2_0627_be229507e65...
1,AAUZebraFish_5,images/AAUZebraFish/data/Vid2_0032_6a32c1f8cff...
2,AAUZebraFish_5,images/AAUZebraFish/data/Vid2_0392_353b7bef601...
3,AAUZebraFish_5,images/AAUZebraFish/data/Vid2_1048_5083ecc3d10...
4,AAUZebraFish_5,images/AAUZebraFish/data/Vid2_1090_6f64a8b87dc...


In [4]:
valid_df = pd.read_csv(VALID_META)[['identity', 'path']].iloc[:160]
valid_df.head()

,identity,path
0,AAUZebraFish_5,images/AAUZebraFish/data/Vid2_0171_b24c0158895...
1,AAUZebraFish_5,images/AAUZebraFish/data/Vid2_1023_0d3e7eaaba5...
2,AAUZebraFish_5,images/AAUZebraFish/data/Vid2_0560_a59bd41df84...
3,AAUZebraFish_5,images/AAUZebraFish/data/Vid2_0202_d3b4df168f9...
4,AAUZebraFish_5,images/AAUZebraFish/data/Vid2_0593_aeff758b1e5...


In [5]:
print(f'train # unique IDs: {train_df["identity"].nunique()}')
print(f'valid # unique IDs: {valid_df["identity"].nunique()}')

train # unique IDs: 2
valid # unique IDs: 2


In [6]:
print(f'train_df shape: {train_df.shape}')
print(f'valid_df shape: {valid_df.shape}')

train_df shape: (1600, 2)
valid_df shape: (160, 2)


In [7]:
train_transform = [
    Lambda(lambda img: img.repeat(3, 1, 1) if img.shape[0] == 1 else img),
    RandomResizedCrop(TRAIN_DIM),
    RandomHorizontalFlip(),
    ColorJitter(BRIGHTNESS, CONTRAST, SATURATION, HUE),
    Normalize(NORM_MEANS, NORM_STDS)
]

valid_transform = [
    Lambda(lambda img: img.repeat(3, 1, 1) if img.shape[0] == 1 else img),
    Resize(VALID_DIM),
    CenterCrop(TRAIN_DIM),
    Normalize(NORM_MEANS, NORM_STDS)
]

In [8]:
valid_dataset = Triplets(valid_df, transform=Compose(valid_transform))
valid_dataloader = DataLoader(dataset=valid_dataset, batch_size=BATCH_SIZE)

In [9]:
model = PyraTCAiT(embed_dims=EMBED_DIMS, head_counts=HEAD_COUNTS, mlp_ratios=MLP_RATIOS, sr_ratios=SR_RATIOS, depths=DEPTHS,
                  num_stages=NUM_STAGES, dropout=DROPOUT, first_patch_dim=PATCH_SIZE, in_channels=CHANNELS, in_dim=TRAIN_DIM, 
                  add_classifier=ADD_CLASSIFIER, use_improved=USE_IMPROVED, classification_intent=CLASSIFICATION_INTENT, num_classes=NUM_CLASSES)

In [10]:
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, betas=tuple(BETAS), weight_decay=WEIGHT_DECAY)
scheduler = WarmupCosineScheduler(optimizer=optimizer, warmup_epochs=WARMUP_EPOCHS, total_epochs=NUM_EPOCHS, eta_min=ETA_MIN)

In [11]:
loss_fn = TCLoss(use_label_smoothing=USE_LABEL_SMOOTHING)

In [12]:
distiller = DataDistiller(df=train_df, transform=train_transform, valid_dataloader=valid_dataloader, model=model, loss_fn=loss_fn, optimizer=optimizer, scheduler=scheduler, nepochs=NUM_EPOCHS, batch_size=BATCH_SIZE, 
                          nclasses=NUM_CLASSES, ntriplets=NUM_TRIPLETS, nworkers=NUM_WORKERS, thetas=THETAS, pretr_model=PRETR_MODEL, checkpoints_dir=CHECKPOINTS_DIR, base_data_dir=BASE_DATA_DIR, embeddings_dir=EMBEDDINGS_DIR,
                          triplets_dir=TRIPLETS_DIR, device=DEVICE, gpu_id=GPU_ID, start_epoch=START_EPOCH, ddp=USE_DDP, disable_progress_bar=DISABLE_PROGRESS_BAR, max_attempts=MAX_ATTEMPTS, p_max=P_MAX, margin=MARGIN, remine_freq=REMINE_FREQ)

Unexpected keys (bn2.bias, bn2.num_batches_tracked, bn2.running_mean, bn2.running_var, bn2.weight, classifier.bias, classifier.weight, conv_head.weight) found while loading pretrained weights. This may be expected if model is being adapted.


In [13]:
next(model.parameters()).device

device(type='cuda', index=0)

In [14]:
distiller._initial_embed(debug=True)
distiller._save_embeddings(epoch=START_EPOCH, metric=0.0)

Initial Embedding, Batch [99/100]: 100%|██████████| 100/100 [00:16<00:00,  5.99it/s]


Epoch 0 Embeddings Saved!


In [15]:
len(list(distiller.embeddings.keys()))

2

In [16]:
len(list(distiller.embeddings['AAUZebraFish_5'].keys()))

978

In [17]:
distiller.embeddings.keys()

dict_keys(['AAUZebraFish_5', 'AAUZebraFish_3'])

In [18]:
distiller._load_embeddings(epoch=START_EPOCH + 1)

Previous Epoch's Embeddings Loaded!


0.0

In [19]:
len(list(distiller.embeddings.keys()))

2

In [20]:
len(list(distiller.embeddings['AAUZebraFish_5'].keys()))

978

In [21]:
distiller.embeddings.keys()

dict_keys(['AAUZebraFish_5', 'AAUZebraFish_3'])

In [22]:
omegas_1 = distiller._omegas(p=0.0, p_max=1.0)

print(f'omegas_1[0] == 0.5? {omegas_1[0] == 0.5}')
print(f'omegas_1[1] == 0.5? {omegas_1[1] == 0.5}')
print(f'omegas_1[2] == 0.0? {omegas_1[2] == 0.0}\n')

print(f'sum(omegas_1) == 1.00? {sum(omegas_1) == 1.00}')

omegas_1[0] == 0.5? True
omegas_1[1] == 0.5? True
omegas_1[2] == 0.0? True

sum(omegas_1) == 1.00? True


In [23]:
omegas_2 = distiller._omegas(p=0.15, p_max=1.0)

print(f'omegas_2[0] == 0.5? {omegas_2[0] == 0.5}')
print(f'omegas_2[1] == 0.5? {omegas_2[1] == 0.5}')
print(f'omegas_2[2] == 0.0? {omegas_2[2] == 0.0}\n')

print(f'sum(omegas_2) == 1.00? {sum(omegas_2) == 1.00}')

omegas_2[0] == 0.5? True
omegas_2[1] == 0.5? True
omegas_2[2] == 0.0? True

sum(omegas_2) == 1.00? True


In [24]:
omegas_3 = distiller._omegas(p=0.3, p_max=1.0)

print(f'omegas_3[0] == 0.5? {omegas_2[0] == 0.5}')
print(f'omegas_3[1] == 0.5? {omegas_2[1] == 0.5}')
print(f'omegas_3[2] == 0.0? {omegas_2[2] == 0.0}\n')

print(f'sum(omegas_3) == 1.00? {sum(omegas_3) == 1.00}')

omegas_3[0] == 0.5? True
omegas_3[1] == 0.5? True
omegas_3[2] == 0.0? True

sum(omegas_3) == 1.00? True


In [25]:
omegas_4 = distiller._omegas(p=0.45, p_max=1.0)

print(f'omegas_4[0] == 0.375? {omegas_4[0] == 0.375}')
print(f'omegas_4[1] == 0.625? {omegas_4[1] == 0.625}')
print(f'omegas_4[2] == 0.0? {omegas_4[2] == 0.0}\n')

print(f'sum(omegas_4) == 1.00? {sum(omegas_4) == 1.00}')

omegas_4[0] == 0.375? True
omegas_4[1] == 0.625? True
omegas_4[2] == 0.0? True

sum(omegas_4) == 1.00? True


In [26]:
omegas_5 = distiller._omegas(p=0.6, p_max=1.0)

print(f'omegas_5[0] == 0.25? {omegas_5[0] == 0.25}')
print(f'omegas_5[1] == 0.75? {omegas_5[1] == 0.75}')
print(f'omegas_5[2] == 0.0? {omegas_5[2] == 0.0}\n')

print(f'sum(omegas_5) == 1.00? {sum(omegas_5) == 1.00}')

omegas_5[0] == 0.25? True
omegas_5[1] == 0.75? True
omegas_5[2] == 0.0? True

sum(omegas_5) == 1.00? True


In [27]:
omegas_6 = distiller._omegas(p=0.8, p_max=1.0)

print(f'omegas_6[0] == 0.125? {omegas_6[0] == 0.125}')
print(f'omegas_6[1] == 0.625? {omegas_6[1] == 0.625}')
print(f'omegas_6[2] == 0.25? {omegas_6[2] == 0.25}\n')

print(f'sum(omegas_6) == 1.00? {sum(omegas_6) == 1.00}')

omegas_6[0] == 0.125? False
omegas_6[1] == 0.625? True
omegas_6[2] == 0.25? False

sum(omegas_6) == 1.00? True


In [28]:
omegas_6

(0.12499999999999997, 0.625, 0.25000000000000006)

In [29]:
omegas_7 = distiller._omegas(p=1.0, p_max=1.0)

print(f'omegas_7[0] == 0.0? {omegas_7[0] == 0.0}')
print(f'omegas_7[1] == 0.5? {omegas_7[1] == 0.5}')
print(f'omegas_7[2] == 0.5? {omegas_7[2] == 0.5}\n')

print(f'sum(omegas_7) == 1.00? {sum(omegas_7) == 1.00}')

omegas_7[0] == 0.0? True
omegas_7[1] == 0.5? True
omegas_7[2] == 0.5? True

sum(omegas_7) == 1.00? True


In [30]:
trip_data = distiller._random_mine(NUM_TRIPLETS)

20 Randomly-Mined Triplets Selected!


In [31]:
for i, triplet in enumerate(trip_data):
    print(f'triplet {i + 1}: tuple of {len(triplet)} elements (3 identities, 3 paths)')
    
print(f'\ntriplet 1 contents: {trip_data[0]}')

triplet 1: tuple of 6 elements (3 identities, 3 paths)
triplet 2: tuple of 6 elements (3 identities, 3 paths)
triplet 3: tuple of 6 elements (3 identities, 3 paths)
triplet 4: tuple of 6 elements (3 identities, 3 paths)
triplet 5: tuple of 6 elements (3 identities, 3 paths)
triplet 6: tuple of 6 elements (3 identities, 3 paths)
triplet 7: tuple of 6 elements (3 identities, 3 paths)
triplet 8: tuple of 6 elements (3 identities, 3 paths)
triplet 9: tuple of 6 elements (3 identities, 3 paths)
triplet 10: tuple of 6 elements (3 identities, 3 paths)
triplet 11: tuple of 6 elements (3 identities, 3 paths)
triplet 12: tuple of 6 elements (3 identities, 3 paths)
triplet 13: tuple of 6 elements (3 identities, 3 paths)
triplet 14: tuple of 6 elements (3 identities, 3 paths)
triplet 15: tuple of 6 elements (3 identities, 3 paths)
triplet 16: tuple of 6 elements (3 identities, 3 paths)
triplet 17: tuple of 6 elements (3 identities, 3 paths)
triplet 18: tuple of 6 elements (3 identities, 3 paths)
t

In [32]:
trip_data, dist_data = distiller._semihard_mine(NUM_TRIPLETS, MARGIN, debug=True)

20 Semi-Hard Negative Mined Triplets Selected!


In [33]:
for i, triplet in enumerate(trip_data):
    print(f'triplet {i + 1}: tuple of {len(triplet)} elements (3 identities, 3 paths)')
    
print(f'\ntriplet 1 contents: {trip_data[0]}')

triplet 1: tuple of 6 elements (3 identities, 3 paths)
triplet 2: tuple of 6 elements (3 identities, 3 paths)
triplet 3: tuple of 6 elements (3 identities, 3 paths)
triplet 4: tuple of 6 elements (3 identities, 3 paths)
triplet 5: tuple of 6 elements (3 identities, 3 paths)
triplet 6: tuple of 6 elements (3 identities, 3 paths)
triplet 7: tuple of 6 elements (3 identities, 3 paths)
triplet 8: tuple of 6 elements (3 identities, 3 paths)
triplet 9: tuple of 6 elements (3 identities, 3 paths)
triplet 10: tuple of 6 elements (3 identities, 3 paths)
triplet 11: tuple of 6 elements (3 identities, 3 paths)
triplet 12: tuple of 6 elements (3 identities, 3 paths)
triplet 13: tuple of 6 elements (3 identities, 3 paths)
triplet 14: tuple of 6 elements (3 identities, 3 paths)
triplet 15: tuple of 6 elements (3 identities, 3 paths)
triplet 16: tuple of 6 elements (3 identities, 3 paths)
triplet 17: tuple of 6 elements (3 identities, 3 paths)
triplet 18: tuple of 6 elements (3 identities, 3 paths)
t

In [34]:
for i, distances in enumerate(dist_data):
    print(f'triplet {i + 1}: AP [{distances["anchor_positive"]:.4f}], NP [{distances["anchor_negative"]:.4f}], AP + Margin [{distances["anchor_positive_margin"]:.4f}] ---> Condition Met? {distances["anchor_positive"] < distances["anchor_negative"] < distances["anchor_positive_margin"]}')

triplet 1: AP [0.0377], NP [0.0869], AP + Margin [1.0377] ---> Condition Met? True
triplet 2: AP [31.7822], NP [31.7827], AP + Margin [32.7822] ---> Condition Met? True
triplet 3: AP [17.7163], NP [17.7281], AP + Margin [18.7163] ---> Condition Met? True
triplet 4: AP [18.4284], NP [18.4822], AP + Margin [19.4284] ---> Condition Met? True
triplet 5: AP [32.7025], NP [32.7149], AP + Margin [33.7025] ---> Condition Met? True
triplet 6: AP [16.3497], NP [17.1273], AP + Margin [17.3497] ---> Condition Met? True
triplet 7: AP [11.4020], NP [11.4132], AP + Margin [12.4020] ---> Condition Met? True
triplet 8: AP [31.9230], NP [31.9241], AP + Margin [32.9230] ---> Condition Met? True
triplet 9: AP [31.6216], NP [31.6222], AP + Margin [32.6216] ---> Condition Met? True
triplet 10: AP [16.2846], NP [17.1229], AP + Margin [17.2846] ---> Condition Met? True
triplet 11: AP [0.0209], NP [0.0547], AP + Margin [1.0209] ---> Condition Met? True
triplet 12: AP [18.4005], NP [18.4013], AP + Margin [19.40

In [35]:
trip_data, dist_data = distiller._hard_mine(NUM_TRIPLETS, debug=True)

20 Hard Negative Mined Triplets Selected!


In [36]:
for i, triplet in enumerate(trip_data):
    print(f'triplet {i + 1}: tuple of {len(triplet)} elements (3 identities, 3 paths)')
    
print(f'\ntriplet 1 contents: {trip_data[0]}')

triplet 1: tuple of 6 elements (3 identities, 3 paths)
triplet 2: tuple of 6 elements (3 identities, 3 paths)
triplet 3: tuple of 6 elements (3 identities, 3 paths)
triplet 4: tuple of 6 elements (3 identities, 3 paths)
triplet 5: tuple of 6 elements (3 identities, 3 paths)
triplet 6: tuple of 6 elements (3 identities, 3 paths)
triplet 7: tuple of 6 elements (3 identities, 3 paths)
triplet 8: tuple of 6 elements (3 identities, 3 paths)
triplet 9: tuple of 6 elements (3 identities, 3 paths)
triplet 10: tuple of 6 elements (3 identities, 3 paths)
triplet 11: tuple of 6 elements (3 identities, 3 paths)
triplet 12: tuple of 6 elements (3 identities, 3 paths)
triplet 13: tuple of 6 elements (3 identities, 3 paths)
triplet 14: tuple of 6 elements (3 identities, 3 paths)
triplet 15: tuple of 6 elements (3 identities, 3 paths)
triplet 16: tuple of 6 elements (3 identities, 3 paths)
triplet 17: tuple of 6 elements (3 identities, 3 paths)
triplet 18: tuple of 6 elements (3 identities, 3 paths)
t

In [37]:
for i, distances in enumerate(dist_data):
    print(f'triplet {i + 1}: AP [{distances["anchor_positive"]:.4f}], NP [{distances["anchor_negative"]:.4f}] ---> Condition Met? {distances["anchor_positive"] > distances["anchor_negative"]}')

triplet 1: AP [0.0274], NP [0.0217] ---> Condition Met? True
triplet 2: AP [13.0254], NP [0.0173] ---> Condition Met? True
triplet 3: AP [25.6769], NP [0.0849] ---> Condition Met? True
triplet 4: AP [15.5603], NP [0.0219] ---> Condition Met? True
triplet 5: AP [16.3197], NP [0.0297] ---> Condition Met? True
triplet 6: AP [16.3485], NP [16.3307] ---> Condition Met? True
triplet 7: AP [13.7644], NP [0.0679] ---> Condition Met? True
triplet 8: AP [0.0156], NP [0.0100] ---> Condition Met? True
triplet 9: AP [27.3841], NP [1.8513] ---> Condition Met? True
triplet 10: AP [18.3272], NP [18.3157] ---> Condition Met? True
triplet 11: AP [0.0219], NP [0.0123] ---> Condition Met? True
triplet 12: AP [17.8546], NP [14.6642] ---> Condition Met? True
triplet 13: AP [27.2388], NP [22.7143] ---> Condition Met? True
triplet 14: AP [33.1880], NP [16.4670] ---> Condition Met? True
triplet 15: AP [31.3382], NP [27.3388] ---> Condition Met? True
triplet 16: AP [0.0175], NP [0.0125] ---> Condition Met? True

In [38]:
tmp_dataloader = distiller._mine(0.8, 1.0, MARGIN, epoch=0, transform=train_transform)

2 Randomly-Mined Triplets Selected!
12 Semi-Hard Negative Mined Triplets Selected!
5 Hard Negative Mined Triplets Selected!
Epoch 0 Triplets Saved!


In [39]:
tmp_df = distiller._load_triplets(epoch=1)

Previous Epoch's Triplets Loaded!


In [40]:
tmp_df

,anchor_id,positive_id,negative_id,anchor_path,positive_path,negative_path
0,AAUZebraFish_3,AAUZebraFish_3,AAUZebraFish_5,/home/hice1/cclark339/scratch/Data/WildlifeReI...,/home/hice1/cclark339/scratch/Data/WildlifeReI...,/home/hice1/cclark339/scratch/Data/WildlifeReI...
1,AAUZebraFish_3,AAUZebraFish_3,AAUZebraFish_5,/home/hice1/cclark339/scratch/Data/WildlifeReI...,/home/hice1/cclark339/scratch/Data/WildlifeReI...,/home/hice1/cclark339/scratch/Data/WildlifeReI...
2,AAUZebraFish_3,AAUZebraFish_3,AAUZebraFish_5,/home/hice1/cclark339/scratch/Data/WildlifeReI...,/home/hice1/cclark339/scratch/Data/WildlifeReI...,/home/hice1/cclark339/scratch/Data/WildlifeReI...
3,AAUZebraFish_3,AAUZebraFish_3,AAUZebraFish_5,/home/hice1/cclark339/scratch/Data/WildlifeReI...,/home/hice1/cclark339/scratch/Data/WildlifeReI...,/home/hice1/cclark339/scratch/Data/WildlifeReI...
4,AAUZebraFish_5,AAUZebraFish_5,AAUZebraFish_3,/home/hice1/cclark339/scratch/Data/WildlifeReI...,/home/hice1/cclark339/scratch/Data/WildlifeReI...,/home/hice1/cclark339/scratch/Data/WildlifeReI...
5,AAUZebraFish_5,AAUZebraFish_5,AAUZebraFish_3,/home/hice1/cclark339/scratch/Data/WildlifeReI...,/home/hice1/cclark339/scratch/Data/WildlifeReI...,/home/hice1/cclark339/scratch/Data/WildlifeReI...
6,AAUZebraFish_3,AAUZebraFish_3,AAUZebraFish_5,/home/hice1/cclark339/scratch/Data/WildlifeReI...,/home/hice1/cclark339/scratch/Data/WildlifeReI...,/home/hice1/cclark339/scratch/Data/WildlifeReI...
7,AAUZebraFish_3,AAUZebraFish_3,AAUZebraFish_5,/home/hice1/cclark339/scratch/Data/WildlifeReI...,/home/hice1/cclark339/scratch/Data/WildlifeReI...,/home/hice1/cclark339/scratch/Data/WildlifeReI...
8,AAUZebraFish_3,AAUZebraFish_3,AAUZebraFish_5,/home/hice1/cclark339/scratch/Data/WildlifeReI...,/home/hice1/cclark339/scratch/Data/WildlifeReI...,/home/hice1/cclark339/scratch/Data/WildlifeReI...
9,AAUZebraFish_5,AAUZebraFish_5,AAUZebraFish_3,/home/hice1/cclark339/scratch/Data/WildlifeReI...,/home/hice1/cclark339/scratch/Data/WildlifeReI...,/home/hice1/cclark339/scratch/Data/WildlifeReI...
